# Segmenting and Clustering Neighborhoods in Toronto | Part-1¶

1.Start by creating a new Notebook for this assignment.

2.Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

3.To create the above dataframe:


. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. .
. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### Scraping Wikipedia page and creating a Dataframe and Transforming the data on Wiki page into pandas dataframe.

Importing Libraries

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Scrape the List of postal codes of Canada

In [12]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [13]:
soup = BeautifulSoup(source, 'xml')

In [14]:
table=soup.find('table')

#### Dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [15]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

#### Search all the postcode, borough, neighborhood 

In [16]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [17]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning


remove rows where Borough is 'Not assigned'

In [24]:
df=df[df['Borough']!='Not assigned']

In [25]:
df[df['Neighborhood']=='Not assigned']=df['Borough']
df.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [26]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [27]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [28]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [29]:
df_merge.drop_duplicates(inplace=True)

In [30]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [31]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
df_merge.shape

(103, 3)